In [184]:
import requests as re
import pandas as pd
import datetime
import altair as alt
import altair_latimes as lat

In [185]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download
Source: [California Water Watch](https://cww.water.ca.gov/), PRISM

Some parameters

In [186]:
today = datetime.date.today()
today

datetime.date(2023, 6, 12)

In [187]:
atlasID="Statewide"
varKey="PPT_YTD"

Query

In [214]:
url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={today}&atlasID={atlasID}&varKey={varKey}'

In [209]:
data = re.get(url).json()['data']

### Parse

In [210]:
data.keys()

dict_keys(['seriesNames', 'series', 'val', 'min', 'avg', 'max', 'yearEndAvg', 'pavg', 'pmax'])

In [211]:
prcp_min=data['min']
prcp_min

10.4

In [212]:
prcp_avg=data['avg']
prcp_avg

19.79

In [213]:
prcp_max=data['max']
prcp_max

35.89

In [194]:
year_end_avg = data['yearEndAvg']
year_end_avg

23.72

Build dataframe

In [195]:
columns = data['seriesNames']

In [196]:
columns.insert(0, "date")

In [197]:
df = pd.DataFrame(data['series'], columns=columns)

In [198]:
df.columns = df.columns.str.lower()

In [199]:
df['date'] = pd.to_datetime(df['date'])

### Plot

In [200]:
gauge = df[
    (~df.current.isna())
]

In [201]:
gauge = gauge[gauge.date == gauge.date.max()]

In [202]:
gauge["record"] = prcp_max

In [203]:
gauge["pct_of_avg"] = (gauge["current"] / gauge["average"]) * 100

In [204]:
alt.Chart(
    pd.melt(gauge, id_vars="date", value_vars=["current","average"])
).mark_bar().encode(
    x="value",
    y=alt.Y("variable", sort="-x"),
    color="variable"
).properties(
    title="Year-to-date precipitation in California compared to average",
    width=600,
    height=200
)

alt.Chart(...)

In [205]:
line_chart = pd.melt(df, id_vars="date", value_vars=["current","average"])

In [206]:
alt.Chart(line_chart).mark_line().encode(
    x="date:T",
    y="value",
    color="variable",
).properties(
    title=f"California precipitation in {today.year} compared to average",
    width=600
)

alt.Chart(...)

### Export

In [207]:
gauge.to_csv("../../data/processed/precipitation/latest.csv", index=False)